In [25]:
import requests
import json
import time
import os

API_KEY = 'ZdpeG8xFeTZRSqqiNnbe6tvcV9OI6FKenEdeuEpMLuUhq41DQ9EX9-71yKGfsIvxW9mYIzqZznCG2AuECKuQVOmGUvti485q_e_dS4OxSi3ZCSqwcABXYhdjyr9oZnYx'
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
URL = 'https://api.yelp.com/v3/businesses/search'
DATABASE_FILE = 'restaurants_data.json'

def fetch_yelp_data(location, offset):
    params = {
        'term': 'restaurants',
        'location': location,
        'limit': 50,
        'offset': offset
    }
    response = requests.get(URL, headers=HEADERS, params=params)
    return response.json()

def write_to_file(data):

    if os.path.exists(DATABASE_FILE):
        with open(DATABASE_FILE, 'r') as file:
            file_data = json.load(file)
            existing_business_ids = {business['id'] for business in file_data['businesses']}
    else:
        file_data = {'businesses': []}
        existing_business_ids = set()

    new_businesses = [business for business in data['businesses'] if business['id'] not in existing_business_ids]

    if new_businesses:
        # Append new businesses to the existing data
        file_data['businesses'].extend(new_businesses)

        # Write updated data back to the file
        with open(DATABASE_FILE, 'w') as file:
            json.dump(file_data, file, indent=4)

locations = [
    "Los Angeles, CA", "San Diego, CA", "San Jose, CA", "San Francisco, CA",
    "Fresno, CA", "Sacramento, CA", "Long Beach, CA", "Oakland, CA",
    "Bakersfield, CA", "Anaheim, CA", "Santa Ana, CA", "Riverside, CA",
    "Stockton, CA", "Chula Vista, CA", "Irvine, CA", "Fremont, CA",
    "San Bernardino, CA", "Modesto, CA", "Oxnard, CA", "Fontana, CA",
    "Moreno Valley, CA", "Huntington Beach, CA", "Glendale, CA",
    "Santa Clarita, CA", "Garden Grove, CA", "Oceanside, CA",
    "Rancho Cucamonga, CA", "Santa Rosa, CA", "Ontario, CA",
    "Elk Grove, CA", "Corona, CA", "Lancaster, CA", "Palmdale, CA",
    "Hayward, CA", "Salinas, CA", "Pomona, CA", "Escondido, CA",
    "Torrance, CA", "Pasadena, CA", "Orange, CA", "Fullerton, CA",
    "Roseville, CA", "Visalia, CA", "Concord, CA", "Thousand Oaks, CA",
    "Simi Valley, CA", "Victorville, CA", "Vallejo, CA", "Berkeley, CA"
]


for location in locations:
    offset = 0

    while True:
        data = fetch_yelp_data(location, offset)
        if not data.get('businesses'):
            print("Not valid data")
            print(data)
            offset += 50
            break

        write_to_file(data)
        offset += 50
        time.sleep(1)

        if offset >= data['total']:
            print("No more data")
            break

    print("Moving to next location")
print("Finished fetching data")

Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Moving to next location
Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Moving to next location
Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Moving to next location
Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Moving to next location
Not valid data
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
Moving to next location
Not valid data
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
Moving to next location
Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went

In [26]:
import pandas as pd
import json
import os

DATABASE_FILE = 'restaurants_data.json'

def parse_yelp_data(json_data):
    restaurants = []
    for business in json_data['businesses']:
        if 'price' in business  and business['location']['address1']:
            restaurant = {
                'name': business['name'],
                'cuisine': ', '.join([category['title'] for category in business['categories']]),
                'city': business['location']['city'],
                'address': business['location']['address1'],
                'price_range': business.get('price', 'N/A'),
                'average_rating': business['rating'],
                'review_count': business['review_count']
            }
            restaurants.append(restaurant)
    return restaurants


def load_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)



json_data = load_json_file(DATABASE_FILE)
parsed_data = parse_yelp_data(json_data)


restaurants_df = pd.DataFrame(parsed_data)
print(restaurants_df.shape)
restaurants_df.head()
    


(12854, 7)


,name,cuisine,city,address,price_range,average_rating,review_count
0,Bottega,"Italian, Pasta Shops, Pizza",San Francisco,1132 Valencia St,$$,4.3,1119
1,Blind Butcher,"New American, Wine Bars",San Francisco,4058 18th St,$$,4.1,376
2,Fog Harbor Fish House,"Seafood, Wine Bars, Cocktail Bars",San Francisco,39 Pier,$$,4.4,10499
3,The Snug,"Cocktail Bars, New American",San Francisco,2301 Fillmore St,$$,4.0,439
4,Horsefeather,"Cocktail Bars, New American",San Francisco,528 Divisadero St,$$,4.0,606


In [27]:
duplicate_rows = restaurants_df[restaurants_df.duplicated()]
duplicate_rows.head()

,name,cuisine,city,address,price_range,average_rating,review_count


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

restaurants_df['combined_features'] = (
    restaurants_df['cuisine'] + ' ' +
    restaurants_df['price_range'] + ' ' +
    restaurants_df['city'] + ' ' +
    restaurants_df['average_rating'].astype(str) + ' ' +
    restaurants_df['review_count'].astype(str)
)

vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(restaurants_df['combined_features'])
similarity_matrix = cosine_similarity(feature_matrix)


In [31]:
def get_recommendations(restaurant_name, df, similarity_matrix, top_n=10):
    try:
        # Find the index of the restaurant
        idx = df[df['name'].str.contains(restaurant_name, case=False, na=False)].index[0]

        # Get the similarity scores for this restaurant
        sim_scores = list(enumerate(similarity_matrix[idx]))

        # Sort the restaurants based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the top_n most similar restaurants
        sim_scores = sim_scores[1:top_n+1]

        # Get the restaurant indices
        restaurant_indices = [i[0] for i in sim_scores]

        # Return the top_n most similar restaurants
        return df.iloc[restaurant_indices][['name', 'cuisine', 'city', 'average_rating', 'price_range']]
    except IndexError:
        return pd.DataFrame(columns=['name', 'cuisine', 'city', 'average_rating', 'price_range'])

# Example usage
recommendations = get_recommendations('Bottega', restaurants_df, similarity_matrix, top_n=5)
recommendations.head()